<a href="https://colab.research.google.com/github/TarnishingColors/web_mining_hate-speech-detection/blob/main/web_mining_project_nlp_tfidf_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import files
uploaded = files.upload()

Saving d.csv to d (1).csv


In [18]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['d.csv']), encoding="utf-8", encoding_errors='ignore')

In [13]:
!pip install emot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 5.4 MB/s eta 0:00:00


In [14]:
import re
import pickle
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS

In [19]:
#custom preprocessing for this specific dataset
df.drop(['count', 'offensive_language','neither','hate_speech'], axis = 1, inplace = True)
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df_first = df["class"].transform(lambda x: x.replace([1,2], "XX"))
df["class"] = df_first.transform(lambda x: x.replace(0,1).replace("XX",0)) #hate-speach = 1 in class everitying else  = 0
display(df)

,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,0,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,0,"you've gone and broke the wrong heart baby, an..."
24780,0,young buck wanna eat!!.. dat nigguh like I ain...
24781,0,youu got wild bitches tellin you lies


In [16]:
import html


# Functions for extracting 'rt_flag' feature
def annotate_rt(text):
    return 1 if len(text.split()) > 0 and text.split()[0] == 'rt' else 0


def delete_rt_info(text):
    if len(text) > 0:
        return text if text.split()[0] != 'rt' else " ".join(text.split()[1:])
    return text


# Function for converting emojis into word
def convert_emojis(text):
    text = html.unescape(text)
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

In [20]:
# Lemmatization
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()


def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN


def lemmatize(text):
    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in tagged]
    return " ".join(lemmatized)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [21]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


stop_words = set(stopwords.words('english'))


def remove_stopwords(tweet):
  tokens_tweet = word_tokenize(tweet)
  tokens_without_sw = " ".join([word for word in tokens_tweet if not word in stop_words])

  return tokens_without_sw

In [25]:
import re

# class for preprocessing
class DataframePreprocessor():
    def __init__(self, func: callable):
        self.func = func
    
    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self
    
    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        df["tweet"], df["rt_flag"] = zip(*df["tweet"].transform(lambda x: self.func(x)))
        return df


# also adds a new column 'rt_flag'
def preprocessing(tweet):
    re_username = '@[\w\-]+'
    re_exclamation = ""
    re_link = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
          '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    re_all_words = "[^a-z ]+"

    tweet = re.sub(re_username,"", tweet)
    tweet = re.sub(re_link,"link", tweet)
    tweet = tweet.lower()
    tweet = convert_emojis(tweet)
    tweet = re.sub(re_all_words," ",tweet)
    tweet = lemmatize(tweet)
    tweet = remove_stopwords(tweet)
    rt_flag = annotate_rt(tweet)
    tweet = delete_rt_info(tweet)

    return tweet, rt_flag

In [23]:
preprocessing(df.iloc[174]['tweet'])

('wish bae confused face get hoe tho', 0)

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer


# class for vectorizing
class TfIdfVectorizer():
    def __init__(self, vectorizer: callable):
        self.vectorizer = vectorizer
    
    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self
    
    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        X = self.vectorizer.fit_transform(df["tweet"])
        new_df = pd.DataFrame(X.toarray(), columns=self.vectorizer.get_feature_names_out())
        new_df["rt_flag"] = df["rt_flag"]
        new_df["label"] = df["class"] # 'class' is the name of label in d.csv, so we'll need to standartize all the datasets
        return new_df


vectorizer = TfidfVectorizer(norm='l2', min_df=0.05, max_df=0.5)

In [26]:
from sklearn.pipeline import Pipeline


df = df[:500]

pipeline = Pipeline([
    ("preprocessing", DataframePreprocessor(preprocessing)),
    ("vectorizing", TfIdfVectorizer(vectorizer))
])

transformed_df = pipeline.fit_transform(df)
display(transformed_df)

,bitch,face,fuck,get,hoe,joy,joyface,like,link,love,nigga,pussy,shit,tear,rt_flag,label
0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,1,0
1,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,1,0
2,0.331000,0.000000,0.608483,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.721240,0.00000,1,0
3,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.0,0.0,0.000000,0.00000,1,0
4,0.417105,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.908858,0.00000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.000000,0.495461,0.000000,0.0,0.0,0.629412,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.59863,0,0
496,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.00000,0,0
497,0.346973,0.571617,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.743548,0.0,0.0,0.000000,0.00000,0,0
498,0.469417,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.882976,0.0,0.000000,0.0,0.0,0.000000,0.00000,0,0


In [4]:
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.2 MB/s eta 0:00:00


In [5]:
from transformers import BertTokenizer
# Specify the pre-trained model name.
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased' # uncased means this tokenizer will also lower-case automatically

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Loading BERT tokenizer...


In [29]:
#only for searching a max len sake, not for a final version
pipeline = Pipeline([
    ("preprocessing", DataframePreprocessor(preprocessing)),
])

transformed_df = pipeline.fit_transform(df)
transformed_df['len'] = transformed_df['tweet'].transform(lambda x: len(x.split()))

In [31]:
max(transformed_df['len'])

29

In [71]:
import torch


class BertVectorizer():
    def __init__(self, tokenizer: callable):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attention_masks = []
    
    def fit(self, df: pd.DataFrame, y=None, **fit_params):
        return self
    
    def transform(self, df: pd.DataFrame, y=None, **transform_params):
        for text in df['tweet']:
            encoded_text = tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=64, #should be looked into later on a full dataset, as calculated above 29 is max len on a slice of 500 rows
                pad_to_max_length=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            self.input_ids.append(encoded_text['input_ids'])
            self.attention_masks.append(encoded_text['attention_mask'])
            
        self.input_ids = torch.cat(self.input_ids, dim=0)
        self.attention_masks = torch.cat(self.attention_masks, dim=0) # if we are going to use sklearn, then it is not needed
        new_df = pd.DataFrame(self.input_ids)
        new_df['label'] = torch.tensor(df['class'].values)
        return new_df

In [72]:
# BERT pipeline
pipeline = Pipeline([
    ("preprocessing", DataframePreprocessor(preprocessing)),
    ("vectorizing", BertVectorizer(tokenizer))
])

transformed_df = pipeline.fit_transform(df)
transformed_df

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,label
0,101,2450,17612,4550,2160,2158,2467,2202,11669,102,...,0,0,0,0,0,0,0,0,0,0
1,101,2879,23755,3147,5939,3654,1040,7962,2919,26450,...,0,0,0,0,0,0,0,0,0,0
2,101,4830,27767,19387,2412,6616,7743,2707,5390,28679,...,0,0,0,0,0,0,0,0,0,0
3,101,2298,2066,25283,4890,102,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,101,4485,2963,2453,2995,2453,8275,2099,7743,2425,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,101,24151,10189,14839,11245,3892,19685,2227,7697,6569,...,0,0,0,0,0,0,0,0,0,0
496,101,12826,1047,11329,22418,102,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
497,101,8816,2047,6180,2344,2047,2227,7743,2293,16477,...,0,0,0,0,0,0,0,0,0,0
498,101,2514,2378,2066,8861,2690,7743,15161,3335,7861,...,0,0,0,0,0,0,0,0,0,0


In [77]:
from sklearn.model_selection import train_test_split

train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    transformed_df[transformed_df.columns.difference(['label'])], transformed_df['label'], test_size=0.2, random_state=42
)
# train_masks, test_masks, _, _ = train_test_split(
#     attention_masks, input_ids, test_size=0.2, random_state=42
# )

In [78]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=3000)
model.fit(train_inputs, train_labels)

LogisticRegression(max_iter=3000)

In [79]:
from sklearn.metrics import accuracy_score

predictions = model.predict(test_inputs)
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.93
